In [34]:
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import HillClimbSearch, BayesianEstimator, BicScore

In [35]:
data = pd.read_csv("WHO_data.csv")
model = BayesianNetwork([
    ("Fever", "Diagnosis"),
    ("Tiredness", "Diagnosis"),
    ("Dry-Cough", "Diagnosis"),
    ("Difficulty-in-Breathing", "Diagnosis"),
    ("Sore-Throat", "Diagnosis"),
    ("Runny-Nose", "Diagnosis")
])

In [36]:
bic = BicScore(data)
hc = HillClimbSearch(data)
best_model = hc.estimate(scoring_method=bic)

model = BayesianNetwork(best_model.edges())
model.fit(data, estimator=BayesianEstimator)

  0%|          | 9/1000000 [00:01<38:18:29,  7.25it/s] 


In [50]:
import pandas as pd

# Function to predict if a person has COVID-19 based on symptoms
def predict_covid(symptoms, model):
    evidence = {node: val for node, val in zip(model.nodes(), symptoms) if val is not None}
    evidence_df = pd.DataFrame([evidence])
    return model.predict_probability(evidence_df)

# Example usage
input_symptoms = [1, 0, 0, 1,None, 1]  # Example input array of symptoms
prob_covid = predict_covid(input_symptoms, model)
prob_covid = pd.DataFrame(prob_covid)

print("Probability of having COVID-19:", prob_covid)  # Access probability using the label "1" for COVID-19


Probability of having COVID-19:    Difficulty-in-Breathing_0  Difficulty-in-Breathing_1
0                   0.999992                   0.000008
